In [2]:
def fromDigits(digits: list[int], b:int)->int:
    """Compute the number given by digits in base b."""
    if b <= 1:
        raise ValueError("Base must be greater than 1")
    n = 0
    for d in digits:
        n = b * n + d
    return n

In [ ]:
import re
import codecs

log_pattern = r'(?P<datetime>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) - (?P<communicator>\w+) - (?P<level>\w+) - (?P<message>.*): \((?P<details>.*)\)'
counter_received = []
def parse_log_file(file):
    cleaned_logs = []
    with open(file, 'r') as f:
        for line in f:
            match = re.match(log_pattern, line)
            if match:
                timestamp = match.group(1)
                communicator = match.group(2)
                log_level = match.group(3)
                message = match.group(4)
                details = match.group(5)

                # Filter out DEBUG messages
                if log_level != 'DEBUG':
                    cleaned_logs.append(
                        {'timestamp': timestamp, 'communicator': communicator, 
                         'level': log_level, 'message': message, 'details': details})
                    if (details[1:-1].split(',')[0]).startswith('\'counter') and (message == "valid msg"):
                        print(details)
                        print(message)
                        inner = details.split("counter:")[1].split(": hello")[0]  # "\\x00\\n\\xbc"
                        print(details.split(":"))
                        print(details.split(":")[1].split(":"))
                        print(inner.encode())
                        real_bytes = codecs.decode(inner.encode(), "unicode_escape").encode('latin1')
                        print(real_bytes)
                        print(list(real_bytes))
                        l = list(real_bytes)
                        counter = fromDigits(l, 255)
                        counter_received.append(counter)
                        print(f"counter: {list(real_bytes)} : {counter}")
            else:
                print("Log entry does not match the expected format:", line)
        return cleaned_logs


# Clean the raw logs
cleaned_logs = parse_log_file("../log/hfreceiver/2025-11-25.log")

# Print cleaned logs
for log in cleaned_logs:
    print(log)

print(f"received {len(counter_received)}, {len(counter_received)/12000} of messages")

Log entry does not match the expected format: 2025-11-25 14:29:56,673 - hfreceiver - DEBUG - debug message

Log entry does not match the expected format: 2025-11-25 14:29:56,673 - hfreceiver - INFO - info message

Log entry does not match the expected format: 2025-11-25 14:29:56,674 - hfreceiver - WARNING - warn message

Log entry does not match the expected format: 2025-11-25 14:29:56,674 - hfreceiver - ERROR - error message

Log entry does not match the expected format: 2025-11-25 14:29:56,675 - hfreceiver - CRITICAL - critical message

b'counter:\x02: hello', 16, -27, -26, 9, 0
valid msg
["b'counter", '\\x02', " hello', 16, -27, -26, 9, 0"]
['\\x02']
b'\\x02'
b'\x02'
[2]
counter: [2] : 2
b'counter:\x07: hello', 16, -28, -24, 9, 0
valid msg
["b'counter", '\\x07', " hello', 16, -28, -24, 9, 0"]
['\\x07']
b'\\x07'
b'\x07'
[7]
counter: [7] : 7
b'counter:\x0b: hello', 16, -28, -28, 9, 0
valid msg
["b'counter", '\\x0b', " hello', 16, -28, -28, 9, 0"]
['\\x0b']
b'\\x0b'
b'\x0b'
[11]
counte

In [3]:
from math import ceil
SF = 7
BW = 125000
PL = 34
H = 0
DE = 0
CR = 1
n_preamble = 8

def calulcate_airtime(SF, BW, PL, H, DE, CR, n_preamble):
    Tsym = 2**SF / BW
    time_preamble = (n_preamble + 4.25)  * Tsym
    payloadSymbNb = 8 + max(ceil((8*PL - 4*SF + 28 + 16 - 20*H) / (4*(SF - 2*DE))) * (CR + 4), 0)
    time_payload = payloadSymbNb * Tsym
    time_packet = time_preamble + time_payload
    return time_packet

    
calulcate_airtime(SF, BW, PL, H, DE, CR, n_preamble)

0.077056